In [1]:
#import data
import pandas as pd
data_base = pd.read_csv("changed_selected_93_08_NIGMS_R01_base_1993ppid.csv",encoding="latin-1")

In [2]:
#check column names
data_base.columns

Index(['NIH.Spending.Categorization', 'Project.Abstract', 'Project.Terms',
       'Project.Title', 'Public.Health.Relevance', 'Administering..IC',
       'Application.ID', 'Award.Notice.Date', 'FOA', 'Project.Number', 'Type',
       'Activity', 'IC', 'Serial.Number', 'Support.Year', 'Suffix',
       'Program.Official.Information', 'Project.Start.Date',
       'Project.End.Date', 'Study.Section', 'Subproject.Number',
       'Contact.PI..Person.ID', 'Contact.PI...Project.Leader',
       'Other.PI.or.Project.Leader.s.', 'Congressional.District', 'Department',
       'DUNS.Number', 'FIPS', 'Latitude', 'Longitude', 'Organization.Name',
       'Organization.City', 'Organization.State', 'Organization.Type',
       'Organization.Zip', 'Organization.Country', 'ARRA.Indicator',
       'Budget.Start.Date', 'Budget.End.Date', 'CFDA.Code',
       'Funding.Mechanism', 'FY', 'Funding.IC', 'FY.Direct.Costs',
       'FY.Indirect.Costs', 'FY.Total.Cost.by.IC',
       'FY.Total.Cost..Sub.Projects.'],
   

In [3]:
#select useful columns
database=data_base[['Project.Number','Type','FY','Project.End.Date','Budget.Start.Date']]
database.columns=['Project_Number','Type','FY','Project_End_Date','Budget_Start_Date']
database.head()

,Project_Number,Type,FY,Project_End_Date,Budget_Start_Date
0,5R01GM046926-02,5,1993,7/31/96,8/1/93
1,1R01GM048560-01,1,1993,12/31/96,1/1/93
2,5R01GM047286-02,5,1993,4/30/96,5/1/93
3,5R01GM036143-07,5,1993,3/31/95,4/1/93
4,1R01GM049321-01,1,1993,7/31/94,8/1/93


In [4]:
#select unique project by identify unique parts in each project numbers
database['Project_Number']=database['Project_Number'].str[6:12].astype(int)

/Users/lijingning/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
#select competing applications and sort data by project number
database=database[(database.Type == 1) | (database.Type == 2) | (database.Type == 7)|(database.Type == 9)]
database=database.sort(['Project_Number','FY'], ascending=True).reset_index(drop=True)
database.head()

/Users/lijingning/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,Project_Number,Type,FY,Project_End_Date,Budget_Start_Date
0,91,2,1996,5/31/00,6/1/96
1,91,2,2000,5/31/04,6/1/00
2,91,2,2004,5/31/08,6/10/04
3,4725,2,1993,11/30/96,12/1/92
4,5147,2,1994,3/31/97,4/1/94


In [6]:
len(database)

6350

In [7]:
database['Project_End_Date'] = pd.to_datetime(database['Project_End_Date'])
database['Budget_Start_Date'] = pd.to_datetime(database['Budget_Start_Date'])

In [8]:
def months_between(date1,date2):
    m1 = date1.year*12 + date1.month
    m2 = date2.year*12 + date2.month
    months = m2 - m1
    if date1.day > date2.day:
        months = months - 1
    if months < 0: 
        months = 0
    return months

In [10]:
#calculate gaps by each row
gap = []
for i in range(len(database)-1):
    n = database.Project_Number.tolist()
    gsd=database.Project_End_Date.tolist()
    ged=database.Budget_Start_Date.tolist()
    if n[i+1]-n[i]==0:
        a=months_between(ged[i+1],gsd[i])
        if a>=12:
            gap.append(a)
        else:
            gap.append(0)
    else:
        if gsd[i]!=2008:
            b=2008-gsd[i].year
            gap.append(b)
        else:
            gap.append(0)
len(gap)

6349

In [11]:
database.tail()

,Project_Number,Type,FY,Project_End_Date,Budget_Start_Date
6345,86639,1,2008,2012-08-31,2008-09-10
6346,86854,9,2008,2012-07-31,2008-09-30
6347,86877,9,2008,2012-06-30,2008-07-01
6348,87137,9,2008,2012-07-31,2008-09-19
6349,87738,1,2008,2012-07-31,2008-09-22


In [12]:
gap.insert(6349,0)
len(gap)

6350

In [13]:
#add gap column
Gap=pd.Series(gap)
database['Gap'] = Gap.values
database.head()

,Project_Number,Type,FY,Project_End_Date,Budget_Start_Date,Gap
0,91,2,1996,2000-05-31,1996-06-01,0
1,91,2,2000,2004-05-31,2000-06-01,0
2,91,2,2004,2008-05-31,2004-06-10,0
3,4725,2,1993,1996-11-30,1992-12-01,12
4,5147,2,1994,1997-03-31,1994-04-01,0


In [14]:
#mark re-entry by each row
re_entry=[]
for i in range(len(database)-1):
    n=database.Project_Number.tolist()
    gsd=database.Project_End_Date.tolist()
    ged=database.Budget_Start_Date.tolist()
    if n[i+1]-n[i]!=0:
        if gsd[i].year<2008:
            re_entry.append(0)
        else:
            re_entry.append(1)
    else:
        re_entry.append(1)
len(re_entry)

6349

In [15]:
re_entry.insert(6349,1)
len(re_entry)

6350

In [17]:
#add re-entry column
Re_entry=pd.Series(re_entry)
database['Re_Entry'] = Re_entry.values
database.head()

,Project_Number,Type,FY,Project_End_Date,Budget_Start_Date,Gap,Re_Entry
0,91,2,1996,2000-05-31,1996-06-01,0,1
1,91,2,2000,2004-05-31,2000-06-01,0,1
2,91,2,2004,2008-05-31,2004-06-10,0,1
3,4725,2,1993,1996-11-30,1992-12-01,12,0
4,5147,2,1994,1997-03-31,1994-04-01,0,1


In [18]:
#create new data frame for only non-zero gaps and re-entry
data_new=database[['Gap','Re_Entry']]
data_new=data_new[data_new.Gap!= 0]
data_new.head()

,Gap,Re_Entry
3,12,0
6,6,0
8,7,0
9,8,0
13,-3,1


In [19]:
len(data_new)

3016

In [20]:
data_new.groupby('Re_Entry').count()

,Gap
Re_Entry,
0,1931
1,1085


In [47]:
data_new.loc[data_new['Gap']==7]

,Gap,Re_Entry
8,7,0
31,7,0
37,7,0
67,7,0
99,7,0
120,7,0
141,7,0
158,7,0
193,7,0
228,7,0


In [21]:
#calculate total re-entry proportion
total_re_entry=883/2814
total_re_entry

0.31378820184790335

In [53]:
#count numbers for every gap lengths
Count=data_new.groupby('Gap').count()

In [61]:
Count_new=Count[4:]
Count_new

,Re_Entry
Gap,
1,185
2,154
3,168
4,165
5,166
6,150
7,153
8,182
9,163


In [94]:
#check re-entry proportion for gap=35
G1=data_new.loc[data_new['Gap'] == 35]
G1.groupby('Re_Entry').count()

,Gap
Re_Entry,
1,7


In [92]:
#loop for re-entry fracton for each gap length
fraction=[]
gaps=Count_new.index.tolist()
for i in gaps:
    g=data_new.loc[data_new['Gap']==i]
    p=g.groupby('Re_Entry').count()
    n=Count_new['Re_Entry'][Count_new.index==i]
    c=pd.Series.tolist(n)[0]
    if len(p.index)==2:
        d=p.iloc[1]['Gap']
        fraction.append(d/c)
    else:
        r = p.index[0]
        if r==0:
            fraction.append(0)
        else:
            fraction.append(1)
fraction

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.11538461538461539,
 0.53846153846153844,
 0.75,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [93]:
len(fraction)

39

In [65]:
len(gaps)

39